In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [24]:
import deepchem as dc
import deepchem
from rdkit import Chem
import numpy as np
import os
import pickle
from deepchem.contrib.torch.pytorch_graphconv import symmetric_normalize_adj
np.random.seed(2017)


# Load Tox21 dataset
save_file = "./tox21_featurized_nxn.pkl"
if not os.path.exists(save_file):
#if 1==1:
    tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21(featurizer="AdjacencyConv")
    with open(save_file, "wb") as f:
        pickle.dump((tox21_tasks, tox21_datasets, transformers), f, protocol=2)
else:
    with open(save_file, "rb") as f:
        tox21_tasks, tox21_datasets, transformers = pickle.load(f)
train_dataset, valid_dataset, test_dataset = tox21_datasets

train_dataset_x = train_dataset.X
valid_dataset_x = valid_dataset.X
test_dataset_x = test_dataset.X

for idx in range(0, len(train_dataset_x)):
    if idx % 100 == 0: print(idx)
    train_dataset_x[idx] = {"g": symmetric_normalize_adj(train_dataset_x[idx][0]), "x": train_dataset_x[idx][1]}

for idx in range(0, len(valid_dataset_x)):
    valid_dataset_x[idx] = {"g": symmetric_normalize_adj(valid_dataset_x[idx][0]), "x": valid_dataset_x[idx][1]}

for idx in range(0, len(test_dataset_x)):
    test_dataset_x[idx] = {"g": symmetric_normalize_adj(test_dataset_x[idx][0]), "x": test_dataset_x[idx][1]}

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400


In [25]:
train_dataset_x[0]["g"].shape

(150, 150)

In [102]:
import deepchem.contrib.torch.pytorch_graphconv
reload(deepchem.contrib.torch.pytorch_graphconv)
from deepchem.contrib.torch.pytorch_graphconvpytorch_graphconv import GraphConvolution, SingleTaskGraphConvolution, MultiTaskGraphConvolution


In [98]:
gc = GraphConvolution(n_conv_layers=1,
                     max_n_atoms=train_dataset_x[0]["g"].shape[0],
                     n_atom_types=train_dataset_x[0]["x"].shape[1],
                     max_valence=4,
                     conv_layer_dims=[128,128,256],
                     n_fc_layers=2,
                     fc_layer_dims=[64, 12],
                     dropout=0.,
                     return_sigmoid=True)

print(gc)

GraphConvolution (
  (conv_ops): ModuleList (
    (0): Sequential (
      (0): Linear (75 -> 128)
      (1): Dropout (p = 0.0)
      (2): LeakyReLU (0.01, inplace)
      (3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True)
    )
  )
  (fc_ops): ModuleList (
    (0): Sequential (
      (0): Linear (128 -> 64)
      (1): Dropout (p = 0.0)
      (2): ReLU (inplace)
      (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): Sequential (
      (0): Linear (64 -> 12)
      (1): Dropout (p = 0.0)
      (2): ReLU (inplace)
      (3): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True)
    )
  )
)


In [99]:
mtgc = MultiTaskGraphConvolution(net=gc, lr=1e-3, weight_decay=0., n_tasks=train_dataset.y.shape[1])

In [101]:
for epoch in range(0, 100):
    print("epoch: %d" %epoch)
    mtgc.train_epoch(train_dataset_x, train_dataset.y)
    mtgc.evaluate(train_dataset_x,
                       valid_dataset_x,
                       train_dataset.y,
                       valid_dataset.y, 
                       transformer=None,
                       batch_size=32)

epoch: 0
TRAIN:
RMSE:
0.53186988363
ROC AUC:
0.723114679447
TEST:
RMSE:
0.528469609187
ROC AUC:
0.675233267089
epoch: 1
TRAIN:
RMSE:
0.598940708157
ROC AUC:
0.755874104233
TEST:
RMSE:
0.590225975884
ROC AUC:
0.711647401725
epoch: 2
TRAIN:
RMSE:
0.562084877244
ROC AUC:
0.78211365085
TEST:
RMSE:
0.556865888178
ROC AUC:
0.726447343016
epoch: 3
TRAIN:
RMSE:
0.642413079526
ROC AUC:
0.78587978098
TEST:
RMSE:
0.634735195488
ROC AUC:
0.737737387723
epoch: 4
TRAIN:
RMSE:
0.638659640579
ROC AUC:
0.803120459635
TEST:
RMSE:
0.629150083298
ROC AUC:
0.769429276589
epoch: 5
TRAIN:
RMSE:
0.582474446838
ROC AUC:
0.81245649018
TEST:
RMSE:
0.572140395229
ROC AUC:
0.772381808464
epoch: 6
TRAIN:
RMSE:
0.704796795508
ROC AUC:
0.801768784871
TEST:
RMSE:
0.695747050214
ROC AUC:
0.773810687118
epoch: 7
TRAIN:
RMSE:
0.685908526704
ROC AUC:
0.823920923992
TEST:
RMSE:
0.677127303167
ROC AUC:
0.772956146258
epoch: 8
TRAIN:
RMSE:
0.659291154141
ROC AUC:
0.818992835694
TEST:
RMSE:
0.65217092742
ROC AUC:
0.7813513479

TRAIN:
RMSE:
0.935343863728
ROC AUC:
0.85891439429
TEST:
RMSE:
0.93542656286
ROC AUC:
0.793173052471
epoch: 74
TRAIN:
RMSE:
0.939686779958
ROC AUC:
0.864057691692
TEST:
RMSE:
0.939869228688
ROC AUC:
0.796912527196
epoch: 75
TRAIN:
RMSE:
0.935859721835
ROC AUC:
0.865996652648
TEST:
RMSE:
0.935587767454
ROC AUC:
0.795789850841
epoch: 76
TRAIN:
RMSE:
0.927668072835
ROC AUC:
0.837019199442
TEST:
RMSE:
0.926876897414
ROC AUC:
0.789698573711
epoch: 77
TRAIN:
RMSE:
0.937896748472
ROC AUC:
0.841826440158
TEST:
RMSE:
0.938453850481
ROC AUC:
0.7903299136
epoch: 78
TRAIN:
RMSE:
0.940171536989
ROC AUC:
0.840430777579
TEST:
RMSE:
0.9405977132
ROC AUC:
0.781582005312
epoch: 79
TRAIN:
RMSE:
0.94313998238
ROC AUC:
0.820048123048
TEST:
RMSE:
0.943101489585
ROC AUC:
0.758384605785
epoch: 80
TRAIN:
RMSE:
0.943944238094
ROC AUC:
0.84294497659
TEST:
RMSE:
0.944225736878
ROC AUC:
0.791365075553
epoch: 81
TRAIN:
RMSE:
0.941252839501
ROC AUC:
0.850100454599
TEST:
RMSE:
0.941178139709
ROC AUC:
0.805032823788
e